# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pathlib import Path

### Importing the dataset

In [2]:
#Access input folder
current_dir = Path.cwd()
parent_dir = current_dir.parent
input_dir1 = Path (parent_dir/"Pressure_profiles_arrival_omitted")
print ("input",input_dir1)

#Access folders inside input folder
input_dir2_train =  [folder_input for folder_input in input_dir1.iterdir() if folder_input.is_dir()]
print ("2",input_dir2_train)

input G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted
2 [WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_1'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_11'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_13'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_15'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_17'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_19'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_21'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_23'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_a

### Preprocessing

In [3]:
# Access each folder inside input folder
dataset_train = pd.DataFrame(columns = ['Charge_mass', 'Standoff_distance', 'Time', 'Pressure', 'Phase'])
df_list_train = []
for folder_train in input_dir2_train:
    
    # Make a list of data file names
    files_train = list(file_train for file_train in folder_train.rglob("*.xlsx") if 2 < int(file_train.stem[1:]) < 14)
    print ("folder", folder_train)
    
    for file_train in files_train:
        df_list_train.append(pd.read_excel(file_train))
        print (file_train.stem)

dataset_train = pd.concat(df_list_train, ignore_index = True)

folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_1
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_11
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_13
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_15
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_17
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_19
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_21
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_23
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pressure_profiles_arrival_omitted\L13_25
P13
P5
P9
folder G:\Chamodi\Machine_Learning\Pressure_history_2\Pr

In [4]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365320 entries, 0 to 365319
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Charge_mass        365320 non-null  float64
 1   Standoff_distance  365320 non-null  int64  
 2   Time               365320 non-null  float64
 3   Pressure           365320 non-null  float64
 4   Phase              365320 non-null  object 
dtypes: float64(3), int64(1), object(1)
memory usage: 13.9+ MB


In [5]:
dataset_train = pd.get_dummies(dataset_train, columns =['Phase'], dtype = np.uint8)
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365320 entries, 0 to 365319
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Charge_mass        365320 non-null  float64
 1   Standoff_distance  365320 non-null  int64  
 2   Time               365320 non-null  float64
 3   Pressure           365320 non-null  float64
 4   Phase_negative     365320 non-null  uint8  
 5   Phase_positive     365320 non-null  uint8  
dtypes: float64(3), int64(1), uint8(2)
memory usage: 11.8 MB


In [6]:
y = dataset_train['Pressure']
X = dataset_train.drop(['Pressure'], axis=1)
print(X.shape, y.shape)

(365320, 5) (365320,)


In [7]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.95, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,0:3] = sc.fit_transform(X_train[:, 0:3])
print (X_train)

[[ 1.12726829 -0.04002874  0.11161011  1.          0.        ]
 [-1.71620862 -1.26271254 -1.46136406  1.          0.        ]
 [ 0.9241628  -0.04002874 -1.60653096  0.          1.        ]
 ...
 [ 0.01018808  1.18265506  0.32051785  1.          0.        ]
 [-0.39602291 -0.04002874  1.27828068  1.          0.        ]
 [ 1.22882104 -1.26271254 -0.58767382  1.          0.        ]]


In [11]:
print ("X_train", len(X_train))

X_train 18266


### Hyperparameter tuning 1 - layers, neurons, activation function

In [12]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ 2
    nodes = first_layer_nodes
    for i in range(1, 4):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [13]:
from tensorflow.keras.optimizers import Adam
def create_model(first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(first_layer_nodes, last_layer_nodes)
    for i in range(1, 4):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.01)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 300, batch_size = 50)

C:\Users\z5379606\AppData\Local\Temp\ipykernel_24168\571177382.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 300, batch_size = 50)


In [15]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(first_layer_nodes = [100, 125, 150, 175, 200, 225,250],
                  last_layer_nodes = [100, 125, 150, 175, 200, 225,250],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

D:\Anaconda3\envs\python3.9\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/300
366/366 [==============================] - 3s 4ms/step - loss: 1789110144.0000 - mae: 19658.5527
Epoch 2/300
366/366 [==============================] - 1s 4ms/step - loss: 728804544.0000 - mae: 10033.2803
Epoch 3/300
366/366 [==============================] - 2s 5ms/step - loss: 691965056.0000 - mae: 8965.1455
Epoch 4/300
366/366 [==============================] - 2s 6ms/step - loss: 677948672.0000 - mae: 9131.8350
Epoch 5/300
366/366 [==============================] - 2s 5ms/step - loss: 657668160.0000 - mae: 8511.7236
Epoch 6/300
366/366 [==============================] - 6s 15ms/step - loss: 688983168.0000 - mae: 9505.3584
Epoch 7/300
366/366 [==============================] - 6s 17ms/step - loss: 668266368.0000 - mae: 8875.4658
Epoch 8/300
366/366 [==============================] - 2s 4ms/step - loss: 654928832.0000 - mae: 8577.8545
Epoch 9/300
366/366 [==============================] - 3s 7ms/step - loss: 650180608.0000 - mae: 8740.8076
Epoch 10/300
366/366 [==========

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [33]:
from sklearn.model_selection import RandomizedSearchCV
# Function to create model, required for KerasRegressor
def create_model(optimizer='adam', learn_rate=0.001):
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learn_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learn_rate)
    elif optimizer == 'nadam':
        optimizer = Nadam(learning_rate=learn_rate)
 # create model
    model = Sequential()
    model.add(Dense(units=175, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(units=138, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    # Compile model
    model.compile(optimizer=optimizer, loss = 'mean_squared_error', metrics = ['mae'])
    return model


In [34]:
#from scikeras.wrappers import KerasRegressor
# create model
model = KR(build_fn=create_model)

# define the grid search parameters
optimizer = ['RMSprop', 'Adam', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(optimizer=optimizer, learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train, batch_size=50, epochs=300)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/300


C:\Users\z5379606\AppData\Local\Temp\ipykernel_24168\1659311045.py:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model)


293/293 [==============================] - 3s 8ms/step - loss: 9619552256.0000 - mae: 91607.6719
Epoch 2/300
293/293 [==============================] - 2s 7ms/step - loss: 1760457344.0000 - mae: 22890.9453
Epoch 3/300
293/293 [==============================] - 2s 7ms/step - loss: 847332736.0000 - mae: 10839.1309
Epoch 4/300
293/293 [==============================] - 2s 6ms/step - loss: 769949568.0000 - mae: 9927.1885
Epoch 5/300
293/293 [==============================] - 2s 7ms/step - loss: 738387392.0000 - mae: 9721.4619
Epoch 6/300
293/293 [==============================] - 1s 4ms/step - loss: 716830208.0000 - mae: 9229.3516
Epoch 7/300
293/293 [==============================] - 2s 5ms/step - loss: 698017664.0000 - mae: 9071.0225
Epoch 8/300
293/293 [==============================] - 2s 5ms/step - loss: 684402112.0000 - mae: 8839.7812
Epoch 9/300
293/293 [==============================] - 1s 5ms/step - loss: 671769792.0000 - mae: 8634.7656
Epoch 10/300
293/293 [======================

KeyboardInterrupt: 

In [35]:
from sklearn.model_selection import RandomizedSearchCV
# Function to create model, required for KerasRegressor
def create_model(optimizer='adam', learn_rate=0.001):
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learn_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learn_rate)
    elif optimizer == 'nadam':
        optimizer = Nadam(learning_rate=learn_rate)
    
    # create model
    model = Sequential()
    model.add(Dense(units=175, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(units=138, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    
    # Compile model
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return model

In [37]:
# Wrap Keras model for scikit-learn compatibility
model = KerasRegressor(build_fn=create_model)

# Define the random search parameters
param_dist = {
    'optimizer': ['RMSprop', 'Adam', 'Nadam'],
    'learn_rate': [0.001, 0.01, 0.1]  # Uniform distribution for learning rate between 0.001 and 0.1
}

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=5, scoring='r2', n_jobs=1)

# Perform random search
random_result = random_search.fit(X_train, y_train, batch_size=50, epochs=300)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

C:\Users\z5379606\AppData\Local\Temp\ipykernel_24168\2393851057.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model)
D:\Anaconda3\envs\python3.9\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  rng = check_random_state(self.random_state)


Epoch 1/300
293/293 [==============================] - 6s 15ms/step - loss: 9692760064.0000 - mae: 92000.8203
Epoch 2/300
293/293 [==============================] - 8s 26ms/step - loss: 1847485824.0000 - mae: 24010.1348
Epoch 3/300
293/293 [==============================] - 1s 5ms/step - loss: 859605888.0000 - mae: 10946.5107
Epoch 4/300
293/293 [==============================] - 1s 4ms/step - loss: 776660928.0000 - mae: 10283.3760
Epoch 5/300
293/293 [==============================] - 2s 6ms/step - loss: 747585728.0000 - mae: 9624.2773
Epoch 6/300
293/293 [==============================] - 2s 5ms/step - loss: 724651328.0000 - mae: 9263.7275
Epoch 7/300
293/293 [==============================] - 2s 7ms/step - loss: 705223680.0000 - mae: 9125.3965
Epoch 8/300
293/293 [==============================] - 2s 6ms/step - loss: 687917312.0000 - mae: 8891.5000
Epoch 9/300
293/293 [==============================] - 1s 5ms/step - loss: 674308224.0000 - mae: 8533.4072
Epoch 10/300
293/293 [=======